In [123]:
import pandas as pd

In [129]:
data = pd.read_csv('train.csv')

In [130]:
data

,id,Marital status,Application mode,Application order,Course,Daytime/evening attendance,Previous qualification,Previous qualification (grade),Nacionality,Mother's qualification,...,Curricular units 2nd sem (credited),Curricular units 2nd sem (enrolled),Curricular units 2nd sem (evaluations),Curricular units 2nd sem (approved),Curricular units 2nd sem (grade),Curricular units 2nd sem (without evaluations),Unemployment rate,Inflation rate,GDP,Target
0,0,1,1,1,9238,1,1,126.0,1,1,...,0,6,7,6,12.428571,0,11.1,0.6,2.02,Graduate
1,1,1,17,1,9238,1,1,125.0,1,19,...,0,6,9,0,0.000000,0,11.1,0.6,2.02,Dropout
2,2,1,17,2,9254,1,1,137.0,1,3,...,0,6,0,0,0.000000,0,16.2,0.3,-0.92,Dropout
3,3,1,1,3,9500,1,1,131.0,1,19,...,0,8,11,7,12.820000,0,11.1,0.6,2.02,Enrolled
4,4,1,1,2,9500,1,1,132.0,1,19,...,0,7,12,6,12.933333,0,7.6,2.6,0.32,Graduate
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76513,76513,1,17,1,9254,1,1,121.0,1,19,...,0,6,8,5,10.600000,0,13.9,-0.3,0.79,Graduate
76514,76514,1,1,6,9254,1,1,125.0,1,1,...,0,6,9,6,13.875000,0,9.4,-0.8,-3.12,Graduate
76515,76515,5,17,1,9085,1,1,138.0,1,37,...,0,5,8,5,11.400000,1,9.4,-0.8,-3.12,Enrolled
76516,76516,1,1,3,9070,1,1,136.0,1,38,...,0,6,0,0,0.000000,0,7.6,2.6,0.32,Dropout


In [131]:
data.Course = data.Course.rank(method='dense', ascending=True).astype(int) - 1
data['Unemployment rate'] = data['Unemployment rate'] / 100
data['Inflation rate'] = data['Inflation rate'] / 100
data['Target'] = data['Target'].map({'Dropout': 0, 'Enrolled': 1, 'Graduate': 2})

In [132]:
y_train = data['Target']
X_train = data.drop(columns=['Target', 'id'])

In [133]:
# from catboost import CatBoostClassifier
# 
# cat_model = CatBoostClassifier(verbose=0)
# cat_model.fit(X_train, y_train)
# 
# cat_train_preds = cat_model.predict_proba(X_train)

In [134]:
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Dense
# from tensorflow.keras.utils import to_categorical
# 
# y_train_categorical = to_categorical(y_train, num_classes=3)
# 
# nn_model = Sequential()
# nn_model.add(Dense(128, input_dim=X_train.shape[1], activation='relu'))
# nn_model.add(Dense(128, activation='relu'))
# nn_model.add(Dense(64, activation='relu'))
# nn_model.add(Dense(32, activation='softmax'))
# nn_model.add(Dense(3, activation='softmax')) 
# 
# 
# nn_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
# 
# from sklearn.preprocessing import StandardScaler
# scaler = StandardScaler()
# X_train_scaled = scaler.fit_transform(X_train)
# X_test_scaled = scaler.transform(X_test)
# 
# nn_model.fit(X_train_scaled, y_train_categorical, epochs=50, batch_size=10, verbose=1)
# 
# nn_train_preds = nn_model.predict(X_train_scaled)


In [135]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.regularizers import l2
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.utils import to_categorical

# Preprocess the data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
y_train_categorical = to_categorical(y_train, num_classes=3)

# Define the model
nn_model = Sequential()
nn_model.add(Dense(128, input_dim=X_train.shape[1], activation='relu', kernel_regularizer=l2(0.01)))
nn_model.add(Dropout(0.5))
nn_model.add(Dense(128, activation='relu', kernel_regularizer=l2(0.01)))
nn_model.add(Dropout(0.5))
nn_model.add(Dense(64, activation='relu', kernel_regularizer=l2(0.01)))
nn_model.add(Dropout(0.5))
nn_model.add(Dense(3, activation='softmax'))

# Compile the model
optimizer = Adam(learning_rate=0.001)
nn_model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Define early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train the model
nn_model.fit(X_train_scaled, y_train_categorical, epochs=50, batch_size=10, verbose=1, validation_split=0.2, callbacks=[early_stopping])

Epoch 1/50
6122/6122 [==============================] - 4s 594us/step - loss: 0.8148 - accuracy: 0.7908 - val_loss: 0.5843 - val_accuracy: 0.8072
Epoch 2/50
6122/6122 [==============================] - 3s 567us/step - loss: 0.6187 - accuracy: 0.8002 - val_loss: 0.5823 - val_accuracy: 0.8126
Epoch 3/50
6122/6122 [==============================] - 4s 595us/step - loss: 0.6148 - accuracy: 0.8012 - val_loss: 0.5769 - val_accuracy: 0.8087
Epoch 4/50
6122/6122 [==============================] - 4s 581us/step - loss: 0.6130 - accuracy: 0.7998 - val_loss: 0.5719 - val_accuracy: 0.8099
Epoch 5/50
6122/6122 [==============================] - 4s 586us/step - loss: 0.6100 - accuracy: 0.8014 - val_loss: 0.5787 - val_accuracy: 0.8025
Epoch 6/50
6122/6122 [==============================] - 4s 605us/step - loss: 0.6092 - accuracy: 0.8003 - val_loss: 0.5616 - val_accuracy: 0.8127
Epoch 7/50
6122/6122 [==============================] - 3s 568us/step - loss: 0.6094 - accuracy: 0.7999 - val_loss: 0.5612 -

In [99]:
X_test = pd.read_csv('test.csv')

X_test.Course = X_test.Course.rank(method='dense', ascending=True).astype(int) - 1
X_test['Unemployment rate'] = X_test['Unemployment rate'] / 100
data['Inflation rate'] = X_test['Inflation rate'] / 100
X_test['Target'] = X_test['Target'].map({'Dropout': 0, 'Enrolled': 1, 'Graduate': 2})

ids = X_test['id']
X_test = X_test.drop(columns=['id'])

nn_train_preds = nn_model.predict(X_test_scaled)

predictions_df = pd.DataFrame({'id': ids, 'prediction': nn_train_preds})

1595/1595 [==============================] - 0s 251us/step


In [100]:
predictions_df

,id,prediction
0,76518,0
1,76519,2
2,76520,2
3,76521,2
4,76522,1
...,...,...
51007,127525,0
51008,127526,0
51009,127527,0
51010,127528,0


In [101]:
predictions_df = predictions_df.rename(columns={'prediction': 'Target'})

In [103]:
predictions_df['Target'] = predictions_df['Target'].map({0: 'Dropout', 1: 'Enrolled', 2:'Graduate'})

In [104]:
predictions_df.to_csv("academic_success_predictions.csv", index=False)